In [ ]:
# 必要なライブラリのインポート
import keras
from keras.datasets import mnist
from keras import backend as K

keras のライブラリなどなど

In [ ]:
# URLのhttps://github.comをhttps://colab.research.google.com/githubに変換する
# Kerasの関数でデータの読み込み。データをシャッフルして学習データと訓練データに分割
(x_train, y_train), (x_test, y_test) = mnist.load_data()

今回はmnist

In [ ]:
# 行列として入力するための加工
batch_size = 128
num_classes = 10
epochs = 20

img_rows, img_cols = 28, 28

28 * 28にリサイズすることとする

In [ ]:
# 場合分け
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)  # shape =(28,28,1)

In [ ]:
# 入力データ
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# ラベルはone-hot encodingを施す
y_train = y_train.astype('int32')
y_test = y_test.astype('int32')
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test =  keras.utils.np_utils.to_categorical(y_test, num_classes)

one-hot encoding:数値化するさいに他の分類は0に，正解には1とするイメージ
例：正解が4なら， 0~3,5~9のラべルは0にして，4だけ1のラベルをつける
これにより，ノード間の重みの学習の際に役立つ

In [ ]:
# 必要なパッケージのインポート
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))  # shape (28,28,1)=>(26,26,32)
model.add(Conv2D(64, (3, 3), activation='relu')) # 畳み込み層 # shape (26,26,32)=>(24,24,64)
model.add(MaxPooling2D(pool_size=(2, 2))) # プーリング層
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu')) # 全結合層
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

畳み込み層とプーリング層のあとに全結合層を入れるのが通常らしい
このレイヤーをモデルの第1層に使うときはキーワード引数input_shape

* Conv2D(filters, kernel_size, strides=(1, 1), activation=None)
* filters: 整数で，出力空間の次元（つまり畳み込みにおける出力フィルタの数） フィルタサイズ H×H 
* kernel_size: 整数か2つの整数からなるタプル/リストで，2次元の畳み込みウィンドウの幅と高さを指定します. 
          単一の整数の場合は全ての次元に対して同じ値を指定します．単一の整数の場合は正方形のカーネルになります．
* activation: 使用する活性化関数の名前

層の構成は基本的には経験則によるものらしい

keras.layers.Dense(units, activation=None)

* units：正の整数，出力空間の次元数

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
          verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
import matplotlib.pyplot as plt
#Accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
#loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()